## Obtaining Data from Wikipedia

In [1]:
pip install beautifulsoup4

     |████████████████████████████████| 122kB 11.7MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [40]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [64]:
DataSource = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(DataSource).text
#print(source)

In [65]:
soup = BeautifulSoup(source, 'html')

In [66]:
mydata = soup.find('table')
#print(mydata)

In [67]:
# Creating a pandas dataframe with three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [68]:
# Populating the pandas DataFrame with all the postcode, borough, neighborhood 
for tr_cell in mydata.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [69]:
df.head(11)

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


# Data Cleaning
Subsetting data to remove all boroughs with **Not assigned**

In [61]:
df=df[df['Borough']!='Not assigned']
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [72]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [73]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [74]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [75]:
df_merge.drop_duplicates(inplace=True)

In [77]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [78]:
df_merge.shape

(180, 3)